In [3]:
## Standard Imports
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import spacy 
import tweepy as tweepy
import csv, json
from spacy.lang.en.stop_words import STOP_WORDS
from textblob import TextBlob
## Importing the OS and JSON Modules
import os

#geolocator = Nominatim(user_agent="UCI_Tweet_Illness_App")
nlp = spacy.load('en_core_web_sm')

## Importing API Credentials

In [1]:
with open('/Users/zach128/Documents/.secret/twitter_api.json') as f:
    login = json.load(f)
    
auth = tweepy.OAuthHandler(login['consumer_key'], login['consumer_secret'])
auth.set_access_token(login['access_token'], login['access_secret'])
 
twitter_api = tweepy.API(auth)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/zach128/Documents/.secret/twitter_api.json'

In [314]:
# this will tell us how many
limits = twitter_api.rate_limit_status()
print('Calls Left: ')
print(limits['resources']['search']['/search/tweets']['remaining'])
reset_time = limits['resources']['search']['/search/tweets']['reset']
dt = datetime.fromtimestamp(reset_time)
print('\nTime Until Reset: ')
print(dt)

Calls Left: 
44

Time Until Reset: 
2023-02-21 17:21:19


In [ ]:
# Quick Test Query

search_words = 'cold OR flu OR pneumonia OR bronchitis since:2022-01-01 -filter:retweets'
location = "39.8,-95.583068847656,2500km"  # Geographical center of the US with 2500km radius
language = "en"
result_type = "recent"

tweets = tweepy.Cursor(twitter_api.search_tweets, 
                        q=search_words, 
                        geocode=location,
                        lang=language).items(1000)

### Filters Tweets and Import into DataFrame

In [308]:
filtered_tweets = []

for tweet in tweets:
    if tweet.place is not None:
        filtered_tweets.append({
            'text': tweet.text,
            'location': tweet.place.full_name,
            'date': tweet.created_at.date()
        })

[{'text': '@AHagadorn @OurCultureSucks @ThisIsKyleR All Kyle knew about them was that they were trying to kill him. That’s all… https://t.co/lZI5n5ezhE', 'location': 'Saratoga Springs, UT', 'date': datetime.date(2023, 2, 22)}, {'text': 'ACCIDENT - HIT AND RUN - COLD at NW 11TH AVE / NW KEARNEY ST, PORT [Portland Police #PP23000048301] 16:51 #pdx911', 'location': 'Portland, OR', 'date': datetime.date(2023, 2, 22)}, {'text': "Jupiter, Venus, and a thin crescent moon on a cold evening. They'll be even closer together tomorrow night. https://t.co/yKbtKxvWf8", 'location': 'Winnipeg, Manitoba', 'date': datetime.date(2023, 2, 22)}, {'text': 'Idk how people sit in Seattle and be happy lol.. shit is cold 9 months outta the year and rainy most depressing sta… https://t.co/1IeOluftZd', 'location': 'Everett, WA', 'date': datetime.date(2023, 2, 22)}, {'text': 'THREAT - COLD at 2400 E POWELL BLVD, GRSM [Gresham Police #PG23000007467] 16:54 #pdx911', 'location': 'Gresham, OR', 'date': datetime.date(2

In [313]:
tweets_with_loc = pd.DataFrame(filtered_tweets)
tweets_with_loc.head()
print(tweets_with_loc.shape)

(50, 3)


In [72]:
df = pd.read_csv('2020_2022_cold_symptoms.csv')
df.head()

,1280615718,2019-06-30 23:45:48+00:00,"Seattle, WA","[-122.436232, 47.4953154, -122.2249728, 47.734319]",1.14547857538816E+018,"I tried putting it off but Iâ€™m gonna have to take some painkillers. Iâ€™d finally detoxed from them because they were giving me massive migraines , but at this point Iâ€™d rather have a splitting headache than my body in spasms."
0,3.807060e+08,2019-06-30 23:44:11+00:00,"McAllen, TX","[-98.2958236, 26.143437, -98.195525, 26.337943]",1.145478e+18,I hope someone ices me when I walk out of the ...
1,3.169978e+08,2019-06-30 23:40:29+00:00,"Council Bluffs, IA","[-95.9235509, 41.194487, -95.746366, 41.300487]",1.145477e+18,"I like my steak rare, my woman bare, with some..."
2,2.827821e+08,2019-06-30 23:38:40+00:00,"Hope Mills, NC","[-79.033596, 34.894886, -78.834267, 35.009696]",1.145477e+18,@QuickClock23 @SmallcageGG @EBischoff @WWE Wel...
3,1.452974e+07,2019-06-30 23:32:54+00:00,"San Diego, CA","[-117.282538, 32.53962, -116.9274403, 33.0804044]",1.145475e+18,Also I cannot remember a more insane free agen...
4,8.469084e+17,2019-06-30 23:32:27+00:00,"Beverly Hills, CA","[-118.4273439, 34.052559, -118.371934, 34.112434]",1.145475e+18,"@ZeeuwsGamertje That was from Friday, I did tw..."


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54345 entries, 0 to 54344
Data columns (total 6 columns):
 #   Column                                                                                                                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                  --------------  -----  
 0   1280615718                                                                                                                                                                                                                              54345 non-null  float64
 1   2019-06-30 23:45:48+00:00                                                                               

In [53]:
#df = df.reset_index().rename(columns={'index': 'Header'})

In [54]:
#df.index = ['User_ID', 'Time', 'Location', 'Coordinates', 'Tweet_ID', 'Tweet']
#print(df)

In [55]:
df.set_axis(["User_ID", "Time", "Location", "Coordinates", "Tweet_ID", "Tweet"],
                    axis=1, inplace=True)

In [56]:
df.head(5)

,User_ID,Time,Location,Coordinates,Tweet_ID,Tweet
0,3.807060e+08,2019-06-30 23:44:11+00:00,"McAllen, TX","[-98.2958236, 26.143437, -98.195525, 26.337943]",1.145478e+18,I hope someone ices me when I walk out of the ...
1,3.169978e+08,2019-06-30 23:40:29+00:00,"Council Bluffs, IA","[-95.9235509, 41.194487, -95.746366, 41.300487]",1.145477e+18,"I like my steak rare, my woman bare, with some..."
2,2.827821e+08,2019-06-30 23:38:40+00:00,"Hope Mills, NC","[-79.033596, 34.894886, -78.834267, 35.009696]",1.145477e+18,@QuickClock23 @SmallcageGG @EBischoff @WWE Wel...
3,1.452974e+07,2019-06-30 23:32:54+00:00,"San Diego, CA","[-117.282538, 32.53962, -116.9274403, 33.0804044]",1.145475e+18,Also I cannot remember a more insane free agen...
4,8.469084e+17,2019-06-30 23:32:27+00:00,"Beverly Hills, CA","[-118.4273439, 34.052559, -118.371934, 34.112434]",1.145475e+18,"@ZeeuwsGamertje That was from Friday, I did tw..."


## Pre-Processing Steps 

- Lowercase all text  
- Tokenize all text
-  Removing stop words
- Lemmatizing tokens

In [57]:
processed_tweets = pd.DataFrame({'text': df['Tweet'].str.lower()})

def tokenize_text(text):
    doc = nlp(text)
    return [token.text for token in doc if not token.is_stop]

processed_tweets['tokens'] = processed_tweets['text'].apply(tokenize_text)

#### NOTE: The tweets are so far pre-processed by being tokenized and having stop words removed.

In [78]:
processed_tweets.head()

,text,tokens,lemmas,time,location,coordinates
0,i hope someone ices me when i walk out of the ...,"[hope, ices, walk, exam, tomorrow, ..............","[hope, ice, walk, exam, tomorrow, ...........,...",2019-06-30 23:44:11+00:00,"McAllen, TX","[-98.2958236, 26.143437, -98.195525, 26.337943]"
1,"i like my steak rare, my woman bare, with some...","[like, steak, rare, ,, woman, bare, ,, ice, co...","[like, steak, rare, ,, woman, bare, ,, ice, co...",2019-06-30 23:40:29+00:00,"Council Bluffs, IA","[-95.9235509, 41.194487, -95.746366, 41.300487]"
2,@quickclock23 @smallcagegg @ebischoff @wwe wel...,"[@quickclock23, @smallcagegg, @ebischoff, @wwe...","[quick, think, suck, truly, piece, shit, !, ro...",2019-06-30 23:38:40+00:00,"Hope Mills, NC","[-79.033596, 34.894886, -78.834267, 35.009696]"
3,also i cannot remember a more insane free agen...,"[remember, insane, free, agent, market, ., itâ...","[remember, insane, free, agent, market, ., itâ...",2019-06-30 23:32:54+00:00,"San Diego, CA","[-117.282538, 32.53962, -116.9274403, 33.0804044]"
4,"@zeeuwsgamertje that was from friday, i did tw...","[@zeeuwsgamertje, friday, ,, streams, ., going...","[friday, ,, stream, ., go, live, tomorrow, smm...",2019-06-30 23:32:27+00:00,"Beverly Hills, CA","[-118.4273439, 34.052559, -118.371934, 34.112434]"


In [59]:
#Lemmatizing tokens

def lemmatize_tokens(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

processed_tweets['lemmas'] = processed_tweets['tokens'].apply(lambda tokens: lemmatize_tokens(tokens))

Removing the mentions from the tokenized tweets. Should reduce the complexity of the tokens and removes unneccessary data.

In [65]:
def remove_mentions(tokens):
    return [token for token in tokens if not token.startswith('@')]

processed_tweets['lemmas'] = processed_tweets['lemmas'].apply(lambda tokens: remove_mentions(tokens))

In [69]:
processed_tweets.head()

,text,tokens,lemmas,time,location,coordinates
0,i hope someone ices me when i walk out of the ...,"[hope, ices, walk, exam, tomorrow, ..............","[hope, ice, walk, exam, tomorrow, ...........,...",2019-06-30 23:44:11+00:00,"McAllen, TX","[-98.2958236, 26.143437, -98.195525, 26.337943]"
1,"i like my steak rare, my woman bare, with some...","[like, steak, rare, ,, woman, bare, ,, ice, co...","[like, steak, rare, ,, woman, bare, ,, ice, co...",2019-06-30 23:40:29+00:00,"Council Bluffs, IA","[-95.9235509, 41.194487, -95.746366, 41.300487]"
2,@quickclock23 @smallcagegg @ebischoff @wwe wel...,"[@quickclock23, @smallcagegg, @ebischoff, @wwe...","[quick, think, suck, truly, piece, shit, !, ro...",2019-06-30 23:38:40+00:00,"Hope Mills, NC","[-79.033596, 34.894886, -78.834267, 35.009696]"
3,also i cannot remember a more insane free agen...,"[remember, insane, free, agent, market, ., itâ...","[remember, insane, free, agent, market, ., itâ...",2019-06-30 23:32:54+00:00,"San Diego, CA","[-117.282538, 32.53962, -116.9274403, 33.0804044]"
4,"@zeeuwsgamertje that was from friday, i did tw...","[@zeeuwsgamertje, friday, ,, streams, ., going...","[friday, ,, stream, ., go, live, tomorrow, smm...",2019-06-30 23:32:27+00:00,"Beverly Hills, CA","[-118.4273439, 34.052559, -118.371934, 34.112434]"


In [67]:
processed_tweets['time'] = df['Time']
processed_tweets['location'] = df['Location']
processed_tweets['coordinates'] = df['Coordinates']

In [76]:
processed_tweets.head()

,text,tokens,lemmas,time,location,coordinates
0,i hope someone ices me when i walk out of the ...,"[hope, ices, walk, exam, tomorrow, ..............","[hope, ice, walk, exam, tomorrow, ...........,...",2019-06-30 23:44:11+00:00,"McAllen, TX","[-98.2958236, 26.143437, -98.195525, 26.337943]"
1,"i like my steak rare, my woman bare, with some...","[like, steak, rare, ,, woman, bare, ,, ice, co...","[like, steak, rare, ,, woman, bare, ,, ice, co...",2019-06-30 23:40:29+00:00,"Council Bluffs, IA","[-95.9235509, 41.194487, -95.746366, 41.300487]"
2,@quickclock23 @smallcagegg @ebischoff @wwe wel...,"[@quickclock23, @smallcagegg, @ebischoff, @wwe...","[quick, think, suck, truly, piece, shit, !, ro...",2019-06-30 23:38:40+00:00,"Hope Mills, NC","[-79.033596, 34.894886, -78.834267, 35.009696]"
3,also i cannot remember a more insane free agen...,"[remember, insane, free, agent, market, ., itâ...","[remember, insane, free, agent, market, ., itâ...",2019-06-30 23:32:54+00:00,"San Diego, CA","[-117.282538, 32.53962, -116.9274403, 33.0804044]"
4,"@zeeuwsgamertje that was from friday, i did tw...","[@zeeuwsgamertje, friday, ,, streams, ., going...","[friday, ,, stream, ., go, live, tomorrow, smm...",2019-06-30 23:32:27+00:00,"Beverly Hills, CA","[-118.4273439, 34.052559, -118.371934, 34.112434]"


Trying to figure out how to get the county from the city and state. Could look into the coordinates as a potential solution.

In [91]:
def sentimentAnalysis(lemmas):
    text = ' '.join(lemmas)
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    return polarity

processed_tweets['polarity'] = processed_tweets['lemmas'].apply(lambda lemmas: sentimentAnalysis(lemmas))

In [315]:
processed_tweets.head(10)

,text,tokens,lemmas,time,location,coordinates,City,State,city,state,polarity
0,i hope someone ices me when i walk out of the ...,"[hope, ices, walk, exam, tomorrow, ..............","[hope, ice, walk, exam, tomorrow, ...........,...",2019-06-30 23:44:11+00:00,"McAllen, TX","[-98.2958236, 26.143437, -98.195525, 26.337943]",McAllen,TX,McAllen,TX,0.000000
1,"i like my steak rare, my woman bare, with some...","[like, steak, rare, ,, woman, bare, ,, ice, co...","[like, steak, rare, ,, woman, bare, ,, ice, co...",2019-06-30 23:40:29+00:00,"Council Bluffs, IA","[-95.9235509, 41.194487, -95.746366, 41.300487]",Council Bluffs,IA,Council Bluffs,IA,-0.083333
2,@quickclock23 @smallcagegg @ebischoff @wwe wel...,"[@quickclock23, @smallcagegg, @ebischoff, @wwe...","[quick, think, suck, truly, piece, shit, !, ro...",2019-06-30 23:38:40+00:00,"Hope Mills, NC","[-79.033596, 34.894886, -78.834267, 35.009696]",Hope Mills,NC,Hope Mills,NC,-0.243056
3,also i cannot remember a more insane free agen...,"[remember, insane, free, agent, market, ., itâ...","[remember, insane, free, agent, market, ., itâ...",2019-06-30 23:32:54+00:00,"San Diego, CA","[-117.282538, 32.53962, -116.9274403, 33.0804044]",San Diego,CA,San Diego,CA,-0.012500
4,"@zeeuwsgamertje that was from friday, i did tw...","[@zeeuwsgamertje, friday, ,, streams, ., going...","[friday, ,, stream, ., go, live, tomorrow, smm...",2019-06-30 23:32:27+00:00,"Beverly Hills, CA","[-118.4273439, 34.052559, -118.371934, 34.112434]",Beverly Hills,CA,Beverly Hills,CA,0.112121
5,when you have a fever of 100.3! why??? ðÿ¤¢ðÿ’©,"[fever, 100.3, !, , ?, ?, ?, ðÿ¤¢ðÿ, ’, ©]","[fever, 100.3, !, , ?, ?, ?, ðÿ¤¢ðÿ, ', ©]",2019-06-30 23:28:17+00:00,"Mishawaka, IN","[-86.216235, 41.621439, -86.090052, 41.727645]",Mishawaka,IN,Mishawaka,IN,0.000000
6,@lyza1165 i have a bad relationship with heat ...,"[@lyza1165, bad, relationship, heat, &, amp, ;...","[bad, relationship, heat, &, amp, ;, cold, pai...",2019-06-30 23:24:59+00:00,"Panama City Beach, FL","[-85.95802, 30.1650609, -85.7860766, 30.266595]",Panama City Beach,FL,Panama City Beach,FL,-0.186161
7,"when in texas, i always have to hit up what, w...","[texas, ,, hit, ,, ,, whataburger, stone, cold...","[texas, ,, hit, ,, ,, whataburger, stone, cold...",2019-06-30 23:23:55+00:00,"Austin, TX","[-97.928935, 30.127892, -97.5805133, 30.5187994]",Austin,TX,Austin,TX,-0.600000
8,cold dichotomy of being memorialized when you ...,"[cold, dichotomy, memorialized, slept, memoria...","[cold, dichotomy, memorialize, slept, memorial...",2019-06-30 23:23:13+00:00,"Delran, NJ","[-74.987262, 39.991857, -74.905837, 40.041621]",Delran,NJ,Delran,NJ,-0.200000
9,i'm going to kill whoever invented the common ...,"[going, kill, invented, common, cold]","[go, kill, invent, common, cold]",2019-06-30 23:20:31+00:00,"Michigan, USA","[-90.4181075, 41.696088, -82.122971, 48.306272]",Michigan,USA,Michigan,USA,-0.450000
